In [98]:
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np
import time
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt

seed = 8731
np.random.seed(seed)
tf.random.set_seed(seed)

batchsize = 128
epoch = 20

#Things to do
* Remember to Normalize your data and create validation split from train set.
* Learn about tf.data, tf.slices and also tf.records

In [99]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_val = x_train[45000:50000]
y_val = y_train[45000:50000]
x_train = x_train[0:45000]
y_train = y_train[0:45000]


In [83]:
# adding gaussian noise to test data
x_test = x_test + np.random.normal(loc=0.0, scale=0.05, size=x_test.shape)

In [100]:
# Data Augumentation, we will rotate the image at 15degree angle, and crop the image to the centre
angles = [15]
crop_size = 24

rotated_images = []
for i in range(len(x_train)):
    for angle in angles:
        rotated_image = tf.keras.preprocessing.image.random_rotation(x_train[i], angle)
        pil_image = Image.fromarray(rotated_image.astype('uint8'))
        width, height = pil_image.size
        left = (width - crop_size) / 2
        top = (height - crop_size) / 2
        right = (width + crop_size) / 2
        bottom = (height + crop_size) / 2
        cropped_image = pil_image.crop((left, top, right, bottom))
        resized_image = cropped_image.resize((32, 32))
        rotated_image = np.asarray(resized_image)
        rotated_image = np.expand_dims(rotated_image, axis=0)
        rotated_images.append(rotated_image)

x_train = tf.concat([x_train] + rotated_images, axis=0)
y_train = tf.concat([y_train]*(len(angles)+1), axis=0)

In [101]:
x_train = tf.cast(x_train, tf.float32)
x_train = tf.reshape(x_train, (-1,32,32,3)) / 255.0
x_val = x_val.astype(np.float32).reshape(-1,32,32,3) / 255.0
x_test = x_test.astype(np.float32).reshape(-1,32,32,3) / 255.0


y_train = tf.one_hot(y_train, depth=10)
y_val = tf.one_hot(y_val, depth=10)
y_test = tf.one_hot(y_test, depth=10)

In [ ]:

print(x_train.shape)
print(x_test.shape)
print(x_val.shape)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024, seed=epoch*(seed)).batch(batchsize)
train_dataset_full = train_dataset.shuffle(buffer_size=1024, seed=epoch*(seed)).batch(len(train_dataset))
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batchsize)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(batchsize)
print(len(train_dataset))
print(len(test_dataset))

In [103]:
class ImageRecognitionCNN(tf.keras.Model):
    
    def __init__(self, num_classes, device='cpu:0', checkpoint_directory=None):
        ''' Define the parameterized layers used during forward-pass, the device
            where you would like to run the computation (GPU, TPU, CPU) on and the checkpoint
            directory.
            
            Args:
                num_classes: the number of labels in the network.
                device: string, 'cpu:n' or 'gpu:n' (n can vary). Default, 'cpu:0'.
                checkpoint_directory: the directory where you would like to save or 
                                      restore a model.
        ''' 
        super(ImageRecognitionCNN, self).__init__()
        
        # Initialize layers
        self.conv1 = tf.keras.layers.Conv2D(512, 3, padding='same', activation=None)
        self.conv2 = tf.keras.layers.Conv2D(256, 3,padding='same', activation=None)
        self.pool1 = tf.keras.layers.MaxPool2D()
        self.conv3 = tf.keras.layers.Conv2D(128, 3, padding='same', activation=None)
        self.conv4 = tf.keras.layers.Conv2D(64, 3, padding='same', activation=None)
        self.conv5 = tf.keras.layers.Conv2D(128, 3, padding='same', activation=None)
        self.conv8 = tf.keras.layers.Conv2D(num_classes, 1, padding='same', activation=None)
        
        # Define the device 
        self.device = device
        
        # Define the checkpoint directory
        self.checkpoint_directory = checkpoint_directory
        self.acc = tf.keras.metrics.Accuracy()

    #@tf.function
    def predict(self, images, training):
        """ Predicts the probability of each class, based on the input sample.
            
            Args:
                images: 4D tensor. Either an image or a batch of images.
                training: Boolean. Either the network is predicting in
                          training mode or not.
        """
        x = self.conv1(images)
        x = tf.nn.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = tf.nn.relu(x)
        x = self.pool1(x)
        x = self.conv3(x)
        x = tf.nn.relu(x)
        x = self.pool1(x)
        x = self.conv4(x)
        x = tf.nn.relu(x)
        x = self.pool1(x)
        x = self.conv5(x)
        x = tf.nn.relu(x)
        x = self.conv8(x)
        x = self.pool1(x)
        x = tf.reshape(x, (-1, 1, 10))
        return x


    @tf.function
    def loss_fn(self, images, target, training):
        """ Defines the loss function used during 
            training.         
        """
        preds = self.predict(images, training)
        loss = tf.nn.softmax_cross_entropy_with_logits(labels=target, logits=preds)
        return loss

    @tf.function
    def grads_fn(self, images, target, training):
        """ Dynamically computes the gradients of the loss value
            with respect to the parameters of the model, in each
            forward pass.
        """
        with tf.GradientTape() as tape:
            loss = self.loss_fn(images, target, training)
        return tape.gradient(loss, self.variables)

    
    @tf.function
    def restore_model(self):
        """ Function to restore trained model.
        """
        with tf.device(self.device):
            # Run the model once to initialize variables
            dummy_input = tf.constant(tf.zeros((1,48,48,1)))
            print('Good Morning')
            dummy_pred = self.predict(dummy_input, training=False)
            # Restore the variables of the model
            saver = tf.Saver(self.variables)
            saver.restore(tf.train.latest_checkpoint
                          (self.checkpoint_directory))
    @tf.function
    def save_model(self, global_step=0):
        """ Function to save trained model.
        """
        tf.Saver(self.variables).save(self.checkpoint_directory, 
                                       global_step=global_step)   
    
    @tf.function
    def compute_accuracy_2(self, images, targets):
        """ Compute the accuracy on the input data.
        """
        with tf.device(self.device):
            
            # Predict the probability of each class
            logits = self.predict(images, training=False)
            # Select the class with the highest probability
            
            logits = tf.nn.softmax(logits)
            logits = tf.reshape(logits, [-1, 10])
            targets = tf.reshape(targets, [-1,10])
            preds = tf.argmax(logits, axis=1)
            goal = tf.argmax(targets, axis=1)
            self.acc.update_state(goal, preds)
            # Compute the accuracy
            result = self.acc.result()
            #result = self.acc.result().numpy()
        return result


    def fit_fc(self, training_data, eval_data, optimizer, num_epochs=500, 
            early_stopping_rounds=10, verbose=10, train_from_scratch=False):
        """ Function to train the model, using the selected optimizer and
            for the desired number of epochs. You can either train from scratch
            or load the latest model trained. Early stopping is used in order to
            mitigate the risk of overfitting the network.
            
            Args:
                training_data: the data you would like to train the model on.
                                Must be in the tf.data.Dataset format.
                eval_data: the data you would like to evaluate the model on.
                            Must be in the tf.data.Dataset format.
                optimizer: the optimizer used during training.
                num_epochs: the maximum number of iterations you would like to 
                            train the model.
                early_stopping_rounds: stop training if the loss on the eval 
                                       dataset does not decrease after n epochs.
                verbose: int. Specify how often to print the loss value of the network.
                train_from_scratch: boolean. Whether to initialize variables of the
                                    the last trained model or initialize them
                                    randomly.
        """ 
    
        if train_from_scratch==False:
            self.restore_model()
        
        # Initialize best loss. This variable will store the lowest loss on the
        # eval dataset.
        best_loss = 999.99
        
        # Initialize classes to update the mean loss of train and eval
        train_loss = tf.keras.metrics.Mean('train_loss')
        eval_loss = tf.keras.metrics.Mean('eval_loss')
        acc_train = tf.keras.metrics.Mean('train_acc')
        acc_val = tf.keras.metrics.Mean('val_acc')
        
        # Initialize dictionary to store the loss history
        self.history = {}
        self.history['train_loss'] = []
        self.history['eval_loss'] = []
        self.history['train_acc'] = []
        self.history['val_acc'] = []
        
        # Begin training
        with tf.device(self.device):
            for i in range(num_epochs):
                # Training with gradient descent
                #training_data_x = training_data.shuffle(buffer_size=1024).batch(128)
                for step, (images, target) in enumerate(training_data):
                    grads = self.grads_fn(images, target, True)
                    optimizer.apply_gradients(zip(grads, self.variables))
                    
                # Compute the loss on the training data after one epoch
                for step, (images, target) in enumerate(training_data):
                    loss = self.loss_fn(images, target, False)
                    accuracy = self.compute_accuracy_2(images,target)
                    acc_train(accuracy)
                    train_loss(loss)
                self.history['train_loss'].append(train_loss.result())
                self.history['train_acc'].append(acc_train.result())
                #self.history['train_loss'].append(train_loss.result().numpy())
                #self.history['train_acc'].append(acc_train.result().numpy())
                # Reset metrics
                
                
                # Compute the loss on the eval data after one epoch
                for step, (images, target) in enumerate(eval_data):
                    loss = self.loss_fn(images, target, False)
                    accuracy = self.compute_accuracy_2(images,target)
                    acc_val(accuracy)
                    eval_loss(loss)
                self.history['eval_loss'].append(eval_loss.result())
                self.history['val_acc'].append(acc_val.result())
                # self.history['eval_loss'].append(eval_loss.result().numpy())
                # self.history['val_acc'].append(acc_val.result().numpy())
                # Reset metrics
                # print(train_loss.result())
                # print('Train loss at epoch %d: ' %(i+1), train_loss.result())
                # print('Train Acc at epoch %d: ' %(i+1), acc_train.result())
                    
                # print('Eval loss at epoch %d: ' %(i+1), eval_loss.result())
                # print('Eval Acc at epoch %d: ' %(i+1), acc_val.result())
                
                train_loss.reset_states()
                acc_train.reset_states()
                eval_loss.reset_states()
                acc_val.reset_states()
                #Print train and eval losses
                if (i==0) | ((i+1)%verbose==0):
                    print('Train loss at epoch %d: ' %(i+1), self.history['train_loss'][-1])
                    print('Train Acc at epoch %d: ' %(i+1), self.history['train_acc'][-1])
                    
                    print('Eval loss at epoch %d: ' %(i+1), self.history['eval_loss'][-1])
                    print('Eval Acc at epoch %d: ' %(i+1), self.history['val_acc'][-1])

                # Check for early stopping
                count = 0 
                if self.history['eval_loss'][-1]<best_loss:
                    best_loss = self.history['eval_loss'][-1]
                    count = early_stopping_rounds
                else:
                    count -= 1
                if count==0:
                    break


    def predict_fc(self, test_data):
      # Initialize classes to update the mean loss of train and eval
        test_loss = tf.keras.metrics.Mean('train_loss')
        acc_test = tf.keras.metrics.Mean('test_acc')
        
        
        # Begin training
        with tf.device(self.device):
          for step, (images, target) in enumerate(test_data):
              loss = self.loss_fn(images, target, False)
              accuracy = self.compute_accuracy_2(images,target)
              acc_test(accuracy)
              test_loss(loss)
        print("test_accuracy %d", acc_test.result())
        print("test_loss %d", test_loss.result())
            
                
                



In [104]:
# Specify the path where you want to save/restore the trained variables.
#sample_data
checkpoint_directory = '/content/models_checkpoints/cifar-10/'

# Use the GPU if available.
device = 'gpu:0'

# Define optimizer.
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=1e-3)

# Instantiate model. This doesn't initialize the variables yet.
model = ImageRecognitionCNN(num_classes=10, device=device, 
                              checkpoint_directory=checkpoint_directory)


In [ ]:
# Train model
model.fit_fc(train_dataset, val_dataset, optimizer, num_epochs=20, 
          early_stopping_rounds=2, verbose=2, train_from_scratch=True)

In [78]:
class_names = ["Air-plane", "Auto-mobile", "Bird", "Cat", "Deer",  "Dog", "Frog", "Horse", "Ship", "Truck"]

from typing import List, Optional, Sequence

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(
    true: np.ndarray,
    pred: np.ndarray,
    labels: Optional[List[str]] = None,
    normalize: str = "true",
    figsize: Sequence[int] = (5, 4),
) -> np.ndarray:
    """Plot confusion matrix
    Args:
        true (numpy.array): true label
        pred (numpy.array): predicted label
        labels (List[str]), default=None] list of label names
        normalize (str, default="true): whether to normalize scores, chosen from "true" or "false"
    Returns:
        fig: figure of confusion matrix
    """
    cm = confusion_matrix(true, pred, normalize=normalize)
    fig = plt.figure(figsize=figsize)
    sns.heatmap(
        cm,
        annot=True,
        cmap="Blues",
        square=True,
        vmin=0,
        vmax=1.0,
        xticklabels=labels,
        yticklabels=labels,
    )
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.title("Normalized confusion matrix")

    plt.close()
    return fig

In [95]:
#Run this only once after the first trail, and don't run for the other trails
average = np.zeros((10000, 1))

In [96]:
preds = []
for step, (images, target) in enumerate(test_dataset):
    logits = model.predict(images, training=False)
    logits = tf.nn.softmax(logits)
    pred = tf.argmax(logits, axis=-1)
    preds.extend(pred.numpy().tolist())
concatenated_tensor = tf.convert_to_tensor(preds, dtype=tf.int64)
class_indices = np.arange(10)
predicted_classes = class_indices[concatenated_tensor.numpy()]
average = np.add(average, predicted_classes)

In [50]:
average = average / 5.0

In [ ]:
#avg_preds = np.mean(preds, axis=0)
y_tested = tf.reshape(y_test, [-1, 10])
y_tested = tf.argmax(y_tested, axis=1)

plot_confusion_matrix(y_tested, average, labels=class_names, figsize=(8, 6))

In [ ]:
model.predict_fc(test_dataset)